In [1]:
#read input.txt which contains shakespeare text

In [2]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
#working with 1M characters
len(text)

1115394

In [4]:
#first 100 characters in the text file
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [5]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(chars)
print(vocab_size)

['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
65


In [6]:
#character level tokenizer
stoi = {s:i for i, s in enumerate(chars)}
itos = {i:s for i, s in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

In [7]:
print(encode("hello!"))
print(decode(encode("hello!")))

[46, 43, 50, 50, 53, 2]
hello!


In [8]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [9]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [10]:
#training data pairs
block_size = 8
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(decode((x[:t+1]).tolist()), "followed by", decode([(y[t]).item()]))

F followed by i
Fi followed by r
Fir followed by s
Firs followed by t
First followed by  
First  followed by C
First C followed by i
First Ci followed by t


In [11]:
torch.manual_seed(1337)
batch_size = 4 # of independent sequences to process in parallel
block_size = 8 #maximum context len for prediction

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))  #random indexes to pick block_size num of characters
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs', xb.shape)
print(xb)
print('target', yb.shape)
print(yb)

inputs torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
target torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])


In [12]:
for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print("input", decode(context.tolist()), "output", decode([target.item()]))

input L output e
input Le output t
input Let output '
input Let' output s
input Let's output  
input Let's  output h
input Let's h output e
input Let's he output a
input f output o
input fo output r
input for output  
input for  output t
input for t output h
input for th output a
input for tha output t
input for that output  
input n output t
input nt output  
input nt  output t
input nt t output h
input nt th output a
input nt tha output t
input nt that output  
input nt that  output h
input M output E
input ME output O
input MEO output :
input MEO: output 

input MEO:
 output I
input MEO:
I output  
input MEO:
I  output p
input MEO:
I p output a


**Starting with the bigram language model**

In [13]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        #nn.Embedding(vocab_size, size of the vector representing each token)

    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx) #B(batch), T(token_size), C(size of 1D vector representing each tozen)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens):
        #idx is (B, T) is array of indicies in the current context 
        for _ in range(max_new_tokens):
            #prediction
            logits, loss = self(idx)
            #focus only on last time step
            logits = logits[:, -1, :] #becomes (B, C)
            #apply softmax to get probabilites
            probs = F.softmax(logits, dim = -1) #dim = (B, C)
            #sample from distribution
            idx_next = torch.multinomial(probs, num_samples=1) #(B, 1)
            #append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim = 1) #(B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
out, loss = m(xb, yb)
print(out.shape)
print(loss.item())


#zero represents uniline character
print(decode(m.generate(idx = torch.zeros((1, 1), dtype = torch.long), max_new_tokens = 100)[0].tolist()))

torch.Size([32, 65])
4.878634929656982

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [14]:
#create a pytorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3) #previously only used SGD(stochastic gradient descent, here using adam)
#adam will adapt the learning rate unlike SGD which has a constant learning rate

In [15]:
batch_size = 32
for steps in range(20000):
    #sample a batch of data
    xb, yb = get_batch('train')

    #evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none = True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.4832067489624023


In [16]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype = torch.long), max_new_tokens = 300)[0].tolist()))





QUCatheesicis bas, angomitouze chifomerawhe ty ag

Doone thesellande west, me fffol, whandiffe IAfithods misue, knild he I:
Whe! toudirer' My ayosbly louroura s m', uthos s reveprthoukerdi't avorure fotemowe.
Whamo es t, tstt g t RTRushy,
WAsbr spr my ou pl y,
Witoft at o s me,
Whabr'the Cicuoman


**SELF ATTENTION**

Self Attention Math

In [17]:
#toy example
torch.manual_seed(1337)
B, T, C = 4, 8, 2
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [18]:
xbow = torch.zeros((B, T, C)) #xbow is bag of words
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1] # (t, C)
        xbow[b, t] = torch.mean(xprev, 0)

In [19]:
x[0]

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

In [20]:
xbow[0] #row 2 is sum of row 1 and row 2, and so on

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [21]:
a = torch.tril(torch.ones(3, 3))
a

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [22]:
a = a/torch.sum(a, 1, keepdim = True)
a

tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])

In [23]:
#easier way to achieve xbow 
wei = torch.tril(torch.ones(T, T))
print('wei', wei.shape)
print('x', x.shape)
wei = wei/torch.sum(wei, 1, keepdim = True)
xbow2 = wei @ x
print('xbow2', xbow2.shape)

wei torch.Size([8, 8])
x torch.Size([4, 8, 2])
xbow2 torch.Size([4, 8, 2])


In [24]:
#third version using softmax
tril = torch.tril(torch.ones((T, T)))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
print('wei', wei.shape)
xbow3 = wei @ x
print('xbox3', xbow3.shape)

wei torch.Size([8, 8])
xbox3 torch.Size([4, 8, 2])


In [25]:
#the goal of this is to have 2nd token to be average of all tokens upto 2, 3rd token to be the average of all tokens upto 3, and so on

In [32]:
#complete single head self-attention operations
B, T, C = 4, 8, 32
x = torch.randn((B, T, C)) #x data 
head_size = 16
key = nn.Linear(C, head_size, bias = False)
query = nn.Linear(C, head_size, bias = False)
value = nn.Linear(C, head_size, bias = False)
k = key(x) # (B, T, 16)
q = query(x) # (B, T, 16)
wei = q@k.transpose(-2, -1) #(B, T, 16) @ (B, 16, T) --. (B, T, T)

tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = wei/head_size**-0.5 #important to keep variance at 1 to prevent wei from becoming like one-hot encoding
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
out.shape

torch.Size([4, 8, 16])